In [ ]:
import os

try:
    if CONFIGURED_DIRS:  # type: ignore
        pass
except:
    PLAYGROUND_DIR = os.getcwd()
    os.chdir("../src")

CONFIGURED_DIRS = True

print(f"Playground directory: {PLAYGROUND_DIR}")  # type: ignore
print(f"Working directory: {os.getcwd()}")

In [ ]:
from generation.roll import Roll
from common.note import Note
from common.chord import Chord, ChordQuality
from common.pitch import Pitch
from common.interval import Interval

from generation.instruments.piano import Piano
from generation.chord_progression import ChordProgression

In [ ]:
key = Pitch.from_str("C")

I = Chord(key, ChordQuality.Maj)
ii = Chord(key + Interval.from_str("2"), ChordQuality.Min)
IV = Chord(key + Interval.from_str("4"), ChordQuality.Maj)
V = Chord(key + Interval.from_str("5"), ChordQuality.Maj)

roll = Roll(
    beats_per_minute=110,
    time_signature=(4, 4),
)
piano = roll.add_instrument("Stride Piano", Piano)
assert type(piano) is Piano

chord_progression = ChordProgression(
    start_time=roll.Time(0, 0),
    end_time=roll.Time(4, 2),
)
chord_progression.add_chords(
    (I, roll.Time(0, 0)),
    (IV, roll.Time(1, 0)),
    (ii, roll.Time(2, 0)),
    (V, roll.Time(3, 0)),
    (I, roll.Time(4, 0)),
    (IV.get_V7(), roll.Time(0, 2)),
    (ii.get_V7(), roll.Time(1, 2)),
    (V.get_V7(), roll.Time(2, 2)),
    (I.get_V7(), roll.Time(3, 2)),
)

final_pitch = I.root.reoctave_near_pitch(Pitch.from_str("C3"))
piano.generate(chord_progression)
piano.notes.add(Note(final_pitch, chord_progression.end_time, roll.Duration(1)))

In [ ]:
OUTPUT_DIR = os.path.join(PLAYGROUND_DIR, "output")
OUTPUT_MIDI_SRC = os.path.join(OUTPUT_DIR, "output.mid")

os.makedirs(OUTPUT_DIR, exist_ok=True)
with open(OUTPUT_MIDI_SRC, "wb") as fout:
    piano.to_midi().writeFile(fout)  # type: ignore

In [ ]:
os.system(f"open -a VLC {OUTPUT_MIDI_SRC}")